In [2]:
#PublicKey Generation
def key_gen(m,n,P,Q,l,e,Po,Qo,E):
    R = m*P+n*Q
    Ec = E
    Pi = Po
    Qi = Qo
   # for i in range(0,1):
    K = (l^(e-1))*R
        ### sage
        #phi = EllipticCurveIsogeny(Ec, K)
        #Ec = phi.codomain()
        #R = phi(R)
        #Pi = phi(Pi)
        #Qi = phi(Qi)
        #### kamil
    Ec, Pi, Qi = Short_Weierstrass_Velu(E, K, Pi, Qi)
    return Ec, Pi, Qi

#PK for initiator
def key_gen_init(mA,nA):
    return key_gen(mA,nA,PA,QA,lA,eA,PB,QB,E)

#PK for responder
def key_gen_resp(mB,nB):
    return key_gen(mB,nB,PB,QB,lB,eB,PA,QA,E)

########################################################################

#Key Derivation from Own Private and Other Public information
#Input: own private key, own order, other EC, other aux points
def key_der(m,n,l,e,E,P,Q):
    R = m*P+n*Q
    Ec = E
    #for i in range(0,1):
    K = (l^(e-1))*R
        #sage
        #phi = EllipticCurveIsogeny(Ec, K)
        #Ec = phi.codomain()
        #R = phi(R)
        #kamil
    Ec, _, _ = Short_Weierstrass_Velu(Ec, K, P, Q)
    key = Short_Weierstrass_j_invariant(Ec)
    return key

#Key Derivation by initiator
def key_der_init(m,n,E,P,Q):
    return key_der(m,n,lA,eA,E,P,Q)

#Key Derivation by responder
def key_der_resp(m,n,E,P,Q):
    return key_der(m,n,lB,eB,E,P,Q)

########################################################################

#Testing and test vectors
def test_KA(mA,nA,mB,nB,E):
    In = key_gen_init(mA,nA)
    Re = key_gen_resp(mB,nB)
    kA = key_der_init(mA,nA,Re[0],Re[1],Re[2])
    kB = key_der_resp(mB,nB,In[0],In[1],In[2])
    if kA == kB:
        print("Shared key:", kA)
    else:
        return "ERROR in KA"

def Elliptic_Point_Order(P):
    i = 1
    while P * i != 0:
        P = P * i
        i += 1
    return i
    
def Short_Weierstrass_j_invariant(E):
    a4 = E.a4()
    a6 = E.a6()
    b4 = 2 * a4
    b6 = 4 * a6
    b8 = a4**2 * -1
    c4 = -24 * b4
    c6 = -216 * b6
    d = -8 * b4**3 -27 * b6**2
    j = c4**3 / d
    return j
    
def Short_Weierstrass_Velu(E, gen, P, Q):
    n = Elliptic_Point_Order(gen)
    i = 1
    g2 = []
    rp = []
    rm = []
    d = E.a4()
    e = E.a6()
    while i != n: #partitioning points
        pnt = i * gen
        if pnt * 2 == 0:
            g2.append(pnt)
        else:
            if pnt not in rp and -pnt not in rp:
                rp.append(pnt)
                rm.append(-pnt)
        i += 1
    S = g2 + rp
    v = 0
    w = 0
    dPx = 0
    dQx = 0
    dPy = 0
    dQy = 0
    for point in S:
        fq = 3 * point[0]^2 + d
        gq = -2 * point[1]
        vq = 0
        if point * 2 == 0:
            vq = fq
        else:
            vq = 2 * fq
        uq = gq ^ 2
        v += vq
        w += (uq + point[0] * vq)
        ############ isogeny part
        dPx += vq / (P[0] - point[0]) + (uq / (P[0] - point[0])**2) 
        dQx += vq / (Q[0] - point[0]) + (uq / (Q[0] - point[0])**2)
        dPy += (uq * 2*P[1] / (P[0] - point[0])**3) + (vq * (P[1] - point[1]) / (P[0] - point[0])**2) - (fq * gq / (P[0] - point[0])**2)
        dQy += (uq * 2*Q[1] / (Q[0] - point[0])**3) + (vq * (Q[1] - point[1]) / (Q[0] - point[0])**2) - (fq * gq / (Q[0] - point[0])**2) 
        ############ point image part
    phi_P = (P[0] + dPx, P[1] - dPy)
    phi_Q = (Q[0] + dQx, Q[1] - dQy)
    #print(dPx)
    d_E = d - 5 * v
    e_E = e - 7 * w
    Ec = EllipticCurve([d_E, e_E])
    return Ec, Ec(phi_P), Ec(phi_Q)

def point_image(P, vq, uq, fq, gq, point):
    dPx = vq / (P[0] - point[0]) + uq / (P[0] - point[0])^2 
    dPy = (uq * 2*P[1] / (P[0] - point[0])**3) + (vq * (P[1] - point[1]) / (P[0] - point[0])**2) - (fq * gq / (P[0] - point[0])**2)
    return dPx, dPy

#Parameters

#Curve
lA = 2
lB = 3
eA = 6
eB = 1
f = 1
p = (lA^eA)*(lB^eB)*f - 1
F.<z> = GF(p^2)
E = EllipticCurve([F(1), F(0)]) #ie. y^2=x^3+x
pt = E(46, 113)

########################################################################

#Points
#PAx = F(2374093068336250774107936421407893885897*z + 2524646701852396349308425328218203569693)
#PAy = F(1944869260414574206229153243510104781725*z + 1309099413211767078055232768460483417201)
#PA = E(PAx,PAy)
#QA = E(F(-PAx),F(z*PAy))

#PBx = F(1556716033657530876728525059284431761206*z + 1747407329595165241335131647929866065215)
#PBy = F(3456956202852028835529419995475915388483*z + 1975912874247458572654720717155755005566)
#PB = E(PBx,PBy)
#QB = E(F(-PBx),F(z*PBy))

PA=E(F(189*z+177), F(174*z+27))
QA=E(F(132*z+170), F(151*z+56))

PB=E(F(31*z+80), F(64*z+179))
QB=E(F(145), F(56*z+163))

#Example, random integer coefficients, not both divisible by lA|lB respectively
mA = 27
nA = 16

mB = 17
nB = 65

test_KA(mA,nA,mB,nB,E)

Shared key: 106


##### Obliczanie rzędu punktu na krzywej

In [1]:
def Elliptic_Point_Order(P):
    i = 1
    while P * i != 0:
        P = P * i
        i += 1
    return i

##### Obliczanie niezmiennika j krzywej w postaci krótkiej Weierstrassa
$y^2 = x^3 + a_{4}x + a_{6}, gdzie\;a_{4}, a_{6} \in \mathbb{F}_{p^2}$<br>
$b_{4}=2a_{4}$,    $b_{6}=4a_{6}$<br>
$c_{4}=-24b_{4}$,    $c_{6}=-216b_{6}$<br>
$d=-8b_{4}^{3}-27b_{6}^{2}$<br>
$j=\dfrac{c_{4}^{3}}{d}$

In [2]:
def Short_Weierstrass_j_invariant(E):
    a4 = E.a4()
    a6 = E.a6()
    b4 = 2 * a4
    b6 = 4 * a6
    c4 = -24 * b4
    c6 = -216 * b6
    d = -8 * b4**3 -27 * b6**2
    j = c4**3 / d
    return j

##### Wzory Velu dla krzywej w postaci krótkiej Weierstrassa
Wejście: Krzywa eliptyczna $E_1: y^2 = x^3 + dx + e$ oraz generator jądra tworzonej izogenii $R$<br> 
1. Dla wszystkich punktów wygenerowanych przez $R$, czyli wszystkich punktów z jądra izogenii: jeśli jest to punkt rzędu $2$ dodajemy go do zbioru $G_{2}$, jeśli nie to do zbioru $R_{P}$ dodajemy ten punkt a do $R_{M}$ jego element przeciwny
2. Obliczamy zbiór $S=R_{P} \cup G_{2}$ 
3. Dla każdego punktu $Q = (x_Q, y_Q) \in S$:<br> 
$f_Q = 3x^2_Q + d$<br>
$g_Q = -2y_Q$<br>
Jeśli $Q$ jest rzędu $2$ to $v_q=f_q$, w przeciwnym wypadku $v_q=2f_q$<br>
$u_Q = (g_Q)^2$<br>
4. Obliczamy wartości współczynników $v$ i $w$<br>
$v = \sum_{Q \in S} v_Q, \text{    } w = \sum_{Q \in S} (u_Q + x_Qv_Q)$
<br>
5. Współczynniki wynikowej krzywej otrzymujemy w następujący sposób:<br>
Wynikowa krzywa: $E_2: y^2 = x^3 + Dx + E$<br>
$D=d-5v$<br>
$E=e-7w$<br>
6. Obraz dowolnego punktu z wejściowej krzywej $(x, y)$ jego obraz $(\alpha, \beta)$ po przejściu przez obliczoną izogenię jest dany wzorem:<br>
$\alpha = x+ \sum_{Q \in S} (\frac{v_Q}{x - x_Q} + \frac{u_Q}{(x - x_Q)^2})$<br>
$\beta = y - \sum_{Q \in S} (u_Q \frac{2y}{(x - x_Q)^3} + v_Q\frac{y - y_Q}{(x - x_Q)^2}- \frac{f_Qg_Q}{(x - x_Q)^2})$

In [4]:
def Short_Weierstrass_Velu(E, gen, P, Q):
    n = Elliptic_Point_Order(gen)
    i = 1
    g2 = []
    rp = []
    rm = []
    d = E.a4()
    e = E.a6()
    while i != n: #partitioning points
        pnt = i * gen
        if pnt * 2 == 0:
            g2.append(pnt)
        else:
            if pnt not in rp and -pnt not in rp:
                rp.append(pnt)
                rm.append(-pnt)
        i += 1
    S = g2 + rp
    v = 0
    w = 0
    dPx = 0
    dQx = 0
    dPy = 0
    dQy = 0
    for point in S:
        fq = 3 * point[0]**2 + d
        gq = -2 * point[1]
        vq = 0
        if point * 2 == 0:
            vq = fq
        else:
            vq = 2 * fq
        uq = gq ^ 2
        v += vq
        w += (uq + point[0] * vq)
        ############ isogeny part
        dPx += vq / (P[0] - point[0]) + (uq / (P[0] - point[0])**2) 
        dQx += vq / (Q[0] - point[0]) + (uq / (Q[0] - point[0])**2)
        dPy += (uq * 2*P[1] / (P[0] - point[0])**3) + (vq * (P[1] - point[1]) / (P[0] - point[0])**2) - (fq * gq / (P[0] - point[0])**2)
        dQy += (uq * 2*Q[1] / (Q[0] - point[0])**3) + (vq * (Q[1] - point[1]) / (Q[0] - point[0])**2) - (fq * gq / (Q[0] - point[0])**2) 
        ############ point image part
    d_E = d - 5 * v
    e_E = e - 7 * w
    Ec = EllipticCurve([d_E, e_E])
    phi_P = (P[0] + dPx, P[1] - dPy)
    phi_Q = (Q[0] + dQx, Q[1] - dQy)
    return Ec, Ec(phi_P), Ec(phi_Q)

##### Procedura generacji klucza z parametrów startowych, opisana w schemacie powyżej
Na potrzeby tej pokazowej implementacji algorytm ma tylko jedną rundę, z tego powodu jako jądro izogenii musimy wykorzystać nie punkt $R$, a punkt $K$, jest to punkt $R$ pomnożony przez odpowiedni skalar, by zredukować jego rząd do $2$ lub $3$. Jest to konieczne, ponieważ izogenie - jeśli chcemy uzyskać jakąś o dużym stopniu - musimy obliczać jako złożenie izogenii o małych stopniach. Dla parametrów pokazowych rząd punktu $R_A$ Alicji wynosi $64$ - czyli wymagane $l_A^{e_A}$, by zredukować go do $2$ mnożymy $R_A$ razy $32$. 

In [5]:
def key_gen(m,n,P,Q,l,e,Po,Qo,E):
    R = m*P+n*Q
    Ec = E
    Pi = Po
    Qi = Qo
    K = (l**(e-1))*R
    Ec, Pi, Qi = Short_Weierstrass_Velu(E, K, Pi, Qi)
    return Ec, Pi, Qi

##### Procedura generacji klucza z parametrów publicznych, wymienionych z drugą stroną, opisana w schemacie powyżej
$K$ służy temu samemu celowi co w komórce powyżej

In [6]:
def key_derivation(m,n,l,e,E,P,Q):
    R = m*P+n*Q
    Ec = E
    K = (l**(e-1))*R
    Ec, _, _ = Short_Weierstrass_Velu(Ec, K, P, Q)
    key = Short_Weierstrass_j_invariant(Ec)
    return key

##### Funkcje pomocnicze do pierwszej części wymiany - ustalenia kluczy publicznych, inicjujący wymianę korzysta z pierwszej funkcji (Alicja), strona odpowiadająca z drugiej (Bob)

In [7]:
def key_gen_Alice(mA,nA):
    return key_gen(mA,nA,PA,QA,lA,eA,PB,QB,E)

def key_gen_Bob(mB,nB):
    return key_gen(mB,nB,PB,QB,lB,eB,PA,QA,E)

##### Funkcje pomocnicze do drugiej części wymiany - ustalenia wspólnego tajnego klucza, inicjujący wymianę korzysta z pierwszej funkcji (Alicja), strona odpowiadająca z drugiej (Bob)

In [8]:
def key_der_Alice(m,n,E,P,Q):
    return key_derivation(m,n,lA,eA,E,P,Q)

def key_der_Bob(m,n,E,P,Q):
    return key_derivation(m,n,lB,eB,E,P,Q)

##### Funkcja symulująca działanie protokołu SIDH

In [9]:
def SIDH(mA,nA,mB,nB,E):
    In = key_gen_Alice(mA,nA)
    Re = key_gen_Bob(mB,nB)
    kA = key_der_Alice(mA,nA,Re[0],Re[1],Re[2])
    kB = key_der_Bob(mB,nB,In[0],In[1],In[2])
    if kA == kB:
        print("Wspolny tajny klucz:", kA)
    else:
        return "ERROR"

##### Przykładowa instancja protkołu dla podanych parametrów


In [ ]:
lA = 2
lB = 3
eA = 6
eB = 1
f = 1
p = (lA**eA)*(lB**eB)*f - 1 #liczba pierwsza p
F = GF(p^2) #ciało skończone
E = EllipticCurve([F(1), F(0)]) #ie. y^2=x^3+x
PA = #zalezy nad jakim cialem, musi byc rzedu lA**eA
QA = #zalezy nad jakim cialem, musi byc rzedu lA**eA
PB = #zalezy nad jakim cialem, musi byc rzedu lB**eB
QB = #zalezy nad jakim cialem, musi byc rzedu lB**eB

mA = 27
nA = 16

mB = 17
nB = 32

SIDH(mA, nA, mB, nB, E)